In [1]:
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import levene

In [2]:
df_dating = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-tolstihin/dating_data.csv' , sep = ',')
df_dating

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1
...,...,...,...,...
14509,476,631,0,0
14510,455,770,1,0
14511,260,6,1,1
14512,885,812,1,1


In [3]:
df_dating.groupby(['is_match']) \
    .agg({'group' : 'count'})

,group
is_match,
0,9671
1,4843


In [4]:
df_dating.groupby(['group']) \
    .agg({'is_match' : 'count'})

,is_match
group,
0,4792
1,9722


In [5]:
df_dating.user_id_1.nunique()

1000

In [6]:
df_dating.user_id_2.nunique()

1000

In [7]:
is_match.info()

NameError: name 'is_match' is not defined

In [8]:
df_dating.query("user_id_1 == 79")

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
284,79,664,1,0
2286,79,207,1,0
5448,79,966,1,0
5625,79,165,1,0
5731,79,767,1,0
6618,79,90,1,0
8161,79,6,1,1
9383,79,115,1,0
11237,79,890,1,1


In [9]:
df_dating.query("user_id_2 == 79")

,user_id_1,user_id_2,group,is_match
897,443,79,1,1
2404,794,79,1,0
3870,310,79,1,0
6640,115,79,1,0
7433,925,79,1,0
7499,418,79,1,1
7968,798,79,1,1
9204,304,79,1,1
10492,814,79,1,1
10746,126,79,1,1


Из данных видно, что один пользователь может попасть к первый айди, так и во второй айди (без пересечений пар, то есть если один пользователь подружился с другим пользователем, то данные не "зеркалятся", не дублируются). 
Поэтому первой гипотезой будет статистические различия в конверсии "мэтча" пользователей из разных групп (при условии нормальности системы сплитования)

Посчитаем конверсию для каждого пользователя в отдельный датасет. Для этого нам необходимо посчитать количество записей с каждым пользователем в полях "user_id_1" и "user_id_2" (знаменатель) и кол-во единиц в поле "is_match" (числитель)

In [10]:
#convertion
data_action = pd.merge(df_dating.groupby(['user_id_1'],as_index=False) \
                    .agg({'group' : 'count'}) \
                    .rename(columns={'group': 'count_action_1'}),
                df_dating.groupby(['user_id_2'],as_index=False) \
                    .agg({'group' : 'count'}) \
                    .rename(columns={'group': 'count_action_2'}),
         left_on='user_id_1',
         right_on='user_id_2') \
.drop('user_id_2', axis=1) \
.rename(columns={'user_id_1': 'user_id'})

In [11]:
data_match = pd.merge(df_dating.groupby(['user_id_1'],as_index=False) \
                .agg({'is_match' : 'sum'}) \
                .rename(columns={'is_match': 'count_match_1'}),
         df_dating.groupby(['user_id_2'],as_index=False) \
                .agg({'is_match' : 'sum'}) \
                .rename(columns={'is_match': 'count_match_2'}),
         left_on='user_id_1',
         right_on='user_id_2') \
.drop('user_id_2', axis=1) \
.rename(columns={'user_id_1': 'user_id'})

In [12]:
convertion = pd.merge(data_action, data_match, on = 'user_id')
convertion

,user_id,count_action_1,count_action_2,count_match_1,count_match_2
0,1,24,13,11,5
1,2,16,23,7,9
2,3,16,9,5,3
3,4,9,8,2,1
4,5,22,12,13,5
...,...,...,...,...,...
995,996,8,6,1,1
996,997,12,14,1,0
997,998,18,18,10,8
998,999,7,13,2,1


In [13]:
convertion['convert'] = (convertion.count_match_1 + convertion.count_match_2)/(convertion.count_action_1 + convertion.count_action_2)

In [14]:
convertion = convertion.drop(['count_action_1','count_action_2','count_match_1','count_match_2'], axis=1)

In [15]:
convertion

,user_id,convert
0,1,0.432432
1,2,0.410256
2,3,0.320000
3,4,0.176471
4,5,0.529412
...,...,...
995,996,0.142857
996,997,0.038462
997,998,0.500000
998,999,0.150000


In [62]:
df_dating.groupby(['user_id_1'], as_index=False) \
    .agg({'group' : 'mean'}) \
    .rename(columns={'user_id_1': 'user_id'})

,user_id,group
0,1,1
1,2,1
2,3,1
3,4,0
4,5,1
...,...,...
995,996,0
996,997,0
997,998,1
998,999,0


In [66]:
convertion = pd.merge(convertion, 
         df_dating.groupby(['user_id_1'], as_index=False) \
            .agg({'group' : 'mean'}) \
            .rename(columns={'user_id_1': 'user_id'}),
         on = 'user_id',how='right')

Выдвигаем нулевую гипотезу о равенстве дисперсий и проверим с помощью критерия Левана

In [75]:
levene(convertion.query("group == 0").convert, convertion.query("group == 1").convert)

LeveneResult(statistic=7.013821816267975, pvalue=0.008215929686873376)

Критерий говорит нам о равенстве дисперсий, что в свою очередь говорит нам о повышении вероятности допустить ошибку первого или второго рода. В этом случае передадим параметр "equal_var=False" в t-тест

In [91]:
stats.ttest_ind(convertion.query("group == 0").convert,convertion.query("group == 1").convert, equal_var=False)

Ttest_indResult(statistic=-38.24265817843151, pvalue=4.415153883535013e-197)

В результате получаем pvalue сильно меньше 0.05, что говорит нам о том, что отвегается нулевая гипотеза о равенстве средних, и новую систему можно вводить для всех пользователей.

In [90]:
#среднее для контрольной группы
convertion.query("group == 0").convert.mean()

0.19354949019794696

In [88]:
#среднее для экспериментальной группы
convertion.query("group == 1").convert.mean()

0.402271375706159

In [96]:
#нижняя граница доверительного интервала для экспериментальной группы
convertion.query("group == 1").convert.mean() - 3*(convertion.query("group == 1").convert.std())/np.sqrt(len(convertion.query("group == 1").convert))

0.39130851674620504

In [99]:
#верхняя граница доверительного интервала для контрольной группы
convertion.query("group == 0").convert.mean() + 3*(convertion.query("group == 0").convert.std())/np.sqrt(len(convertion.query("group == 0").convert))

0.20571118380516284

Исходя из "размаха" 3 сигм для наших групп, можно сделать вывод о том, что случайным образом не могла так сильно повыситься доля мэтчей, что говорит об эффективности новой системы.